In [25]:
import pandas as pd
# from graph_part import train_test_validation_split, stratified_k_fold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, f1_score, matthews_corrcoef, log_loss, mean_squared_error, balanced_accuracy_score
from math import sqrt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.utils.class_weight import compute_class_weight
import subprocess
import os 
import shutil
from concurrent.futures import ProcessPoolExecutor, as_completed
def unix_call(command):
    job = subprocess.run(command.split(), shell=True, check=True) 

In [2]:
df = pd.read_csv("../data/processed/model_data_pharokka.csv")
df = pd.read_csv("../data/processed/model_data2.csv")
print(df.shape)
df.head()

(11154, 18)


,Unnamed: 0,id,genome_length,gc_%,sequence,reverse_complement,cds_number,positive_strand_%,negative_strand_%,coding_capacity,trna_count,molecule_type_DNA,molecule_type_RNA,molecule_type_ss-DNA,molecule_type_ss-RNA,jumbophage,topology_circular,topology_linear
0,0,MN335248,7045,60.298,AGTACCGCCCGAATTTCGCAGCAACCCAACCGACGCAAGCCCAACC...,CACCCCAGTTCGATTCCGGGGGGAATGGGGGGTACGCCCCCCACGC...,14,85.71,14.29,104.22995,0,False,False,True,False,0,False,True
1,1,MK250029,540217,25.796,GGACAAAGTTTAAAATCAAGAATTGATAGAAAAACATTTAATAGCA...,TAGATTGAATAATTAAGCAAGGAGTTCATTTAGATGAGATAGTTAG...,0,0.00,0.00,0.00000,0,True,False,False,False,1,True,False
2,2,MK250028,550053,26.012,TTACTCATAACTTATCTTACCCATTCATTAATATAACCATATTCAT...,TTTATTTTAAAATTGTCATATATAACGTTCTCTATTATCTTGTATA...,0,0.00,0.00,0.00000,0,True,False,False,False,1,True,False
3,3,MK250027,551627,26.022,TTACTCATAACTTATCTTACCCATTCATTAATATAACCATATTCAT...,TGCATTACTTATACCAATATAATTACTTACCCATTCTATTCAATAC...,0,0.00,0.00,0.00000,0,True,False,False,False,1,True,False
4,4,MK250026,550702,26.020,TTACTCATAACTTATCTTACCCATTCATTAATATAACCATATTCAT...,TTTATTTTAAAATTGTCATATATAACGTTCTCTATTATCTTGTATA...,0,0.00,0.00,0.00000,0,True,False,False,False,1,True,False


Obtain sequences to process the distances

In [ ]:
#  df = df.iloc[0:50]
genomes = {}

# Iterate over the DataFrame rows
for index, row in df.iterrows():
    genomes[row['id']] = row['sequence']

genomes

# Processing distances with MASH

In [ ]:
# Returns dictionary as output
def unix_call(command):
    # Execute a Unix command and return its stdout
    result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return result.stdout.decode('utf-8')  # Decode stdout to string

def process_genomes(genomes, create_sketch=True):
    temp_dir = 'temp_mash'
    os.makedirs(temp_dir, exist_ok=True)  # Create a temp directory

    # Keep track of the created sketch files
    sketch_files = []  

    # Write the genomes to individual .fna files
    for genome_id, sequence in genomes.items():
        file_path = os.path.join(temp_dir, f"{genome_id}.fna")
        with open(file_path, 'w') as file:
            file.write(f">{genome_id}\n{sequence}\n")  # Write in FASTA format

        if create_sketch:
            # Mash expects the prefix for the output file, it appends '.msh' itself
            sketch_prefix = os.path.join(temp_dir, genome_id)  # Define the prefix
            unix_call(f"mash sketch -o {sketch_prefix} {file_path}")  # Run mash sketch
            sketch_files.append(sketch_prefix + '.msh')  # Store the correct path
        else:
            sketch_files.append(file_path)  # Store the .fna file path if not sketching

    # Calculate the distances between all pairs of genomes
    distances = {}  
    for i, file_1 in enumerate(sketch_files):
        for file_2 in sketch_files[i + 1:]:
            # Construct the mash dist command
            dist_command = f"mash dist {file_1} {file_2}"
            # Execute mash dist and capture the output
            dist_output = unix_call(dist_command)
            distances[(file_1, file_2)] = dist_output.strip()

    # Print the distances
    for pair, dist in distances.items():
        print(f"Distance between {pair[0]} and {pair[1]}:\n{dist}")

    # Remove the temp directory after processing
    # shutil.rmtree(temp_dir)

    return distances

# Call the process_genomes function and print the returned distances
calculated_distances = process_genomes(genomes, create_sketch=False)  # Change to False if not sketching
calculated_distances

Single thread

In [ ]:
# Returns dataframe as output

def unix_call(command):
    # Execute a Unix command and return its stdout
    result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return result.stdout.decode('utf-8')  # Decode stdout to string

def process_genomes(genomes, create_sketch=True):
    temp_dir = 'temp_mash'
    os.makedirs(temp_dir, exist_ok=True)

    sketch_files = []
    for genome_id, sequence in genomes.items():
        file_path = os.path.join(temp_dir, f"{genome_id}.fna")
        with open(file_path, 'w') as file:
            file.write(f">{genome_id}\n{sequence}\n")

        if create_sketch:
            sketch_prefix = os.path.join(temp_dir, genome_id)
            unix_call(f"mash sketch -o {sketch_prefix} {file_path}")
            sketch_files.append(sketch_prefix + '.msh')
        else:
            sketch_files.append(file_path)

    # Prepare a list to hold all distance results
    distance_results = []

    for i, file_1 in enumerate(sketch_files):
        for file_2 in sketch_files[i + 1:]:
            dist_output = unix_call(f"mash dist {file_1} {file_2}")
            for line in dist_output.strip().split('\n'):
                if line:  # Avoid processing empty lines
                    parts = line.split()
                    genome1_id = os.path.basename(parts[0]).replace('.fna', '').replace('.msh', '')
                    genome2_id = os.path.basename(parts[1]).replace('.fna', '').replace('.msh', '')
                    # Add a dictionary for each line of output with the relevant information
                    distance_results.append({
                        'Genome1': genome1_id,
                        'Genome2': genome2_id,
                        'Distance': parts[2],
                        'P-value': parts[3],
                        'Shared-hashes': parts[4]
                    })

    # Convert the list of dictionaries into a DataFrame
    distances_df = pd.DataFrame(distance_results)

    shutil.rmtree(temp_dir)

    return distances_df


# distances_df = process_genomes(genomes, create_sketch=False)
distances_df

Multi thread

In [ ]:
def unix_call(command):
    result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return result.stdout.decode('utf-8')  # Decode stdout to string

def mash_dist_pair(file_1, file_2):
    command = f"mash dist {file_1} {file_2}"
    try:
        dist_output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        result = dist_output.decode('utf-8').strip()
    except subprocess.CalledProcessError as e:
        result = f"Error: {e.output.decode()}"
    # Extract only the file names without the '.fna' or '.msh' extensions
    return (os.path.splitext(os.path.basename(file_1))[0],
            os.path.splitext(os.path.basename(file_2))[0],
            *result.split('\t')[2:])  # Split and unpack the distance, p-value, and shared hashes

def process_genomes(genomes, create_sketch=True):
    temp_dir = 'temp_mash'
    sketch_files = []
    
    # Check if temp_dir already exists and has files
    if os.path.exists(temp_dir) and os.path.isdir(temp_dir):
        # Check if the directory has sketch files or fasta files depending on create_sketch flag
        existing_files = os.listdir(temp_dir)
        for genome_id in genomes.keys():
            expected_file = f"{genome_id}.msh" if create_sketch else f"{genome_id}.fna"
            if expected_file in existing_files:
                sketch_files.append(os.path.join(temp_dir, expected_file))
            else:
                # If expected files are not found, they need to be created
                print(f"Expected file {expected_file} not found in {temp_dir}.")
                # Set flag to indicate file creation is required
                create_files = True
                break
        else:
            # Set flag to indicate no file creation is required if all files were found
            create_files = False
    else:
        # If the directory doesn't exist, create it and set flag to create files
        os.makedirs(temp_dir, exist_ok=True)
        create_files = True

    if create_files:
        # Create FASTA or sketch files if needed
        for genome_id, sequence in genomes.items():
            file_path = os.path.join(temp_dir, f"{genome_id}.fna")
            with open(file_path, 'w') as file:
                file.write(f">{genome_id}\n{sequence}\n")
            
            if create_sketch:
                sketch_prefix = file_path.replace('.fna', '')  # Remove the '.fna' for mash output prefix
                unix_call(f"mash sketch -o {sketch_prefix} {file_path}")
                sketch_files.append(sketch_prefix + '.msh')  # Append the .msh path
            else:
                sketch_files.append(file_path)  # Append the .fna path

    pairs = [(sketch_files[i], sketch_file) for i in range(len(sketch_files)) for sketch_file in sketch_files[i + 1:]]
    distance_results = []

    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(mash_dist_pair, pair[0], pair[1]) for pair in pairs]
        for future in as_completed(futures):
            file_1, file_2, distance, p_value, shared_hashes = future.result()
            if not distance.startswith("Error"):  # Only append if no error occurred
                distance_results.append({
                    'Genome1': file_1,
                    'Genome2': file_2,
                    'Distance': distance,
                    'P-value': p_value,
                    'Shared-hashes': shared_hashes
                })

    distance_df = pd.DataFrame(distance_results)
    shutil.rmtree(temp_dir)  # Clean up temporary directory
    return distance_df

# distances_df2 = process_genomes(genomes, create_sketch=True)
distances_df2

Multiprocess writing into csv to avoid RAM crash

In [30]:
df = pd.read_csv("../data/processed/model_data_test.csv")
# df = df.iloc[0:100]
genomes = {}

# Iterate over the DataFrame rows
for index, row in df.iterrows():
    genomes[row['id']] = row['sequence_inphared']



In [32]:
def unix_call(command):
    result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return result.stdout.decode('utf-8')  # Decode stdout to string

def mash_dist_pair(file_1, file_2):
    command = f"mash dist {file_1} {file_2}"
    try:
        dist_output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        result = dist_output.decode('utf-8').strip()

        # Parse the output to extract shared hashes and total hashes
        parts = result.split('\t')  # Split the mash output into parts
        shared_hashes_info = parts[4]  # Get the shared hashes information, e.g., "1/1000"
        shared, total = map(int, shared_hashes_info.split('/'))  # Split into shared and total hashes
        identity = (shared / total) * 100 if total > 0 else 0  # Calculate identity percentage

        # Return the result with identity percentage added
        return (os.path.splitext(os.path.basename(file_1))[0],  # Genome1 ID
                os.path.splitext(os.path.basename(file_2))[0],  # Genome2 ID
                parts[2],  # Distance
                parts[3],  # P-value
                shared_hashes_info,  # Shared-hashes
                identity)  # Identity percentage

    except subprocess.CalledProcessError as e:
        error_message = f"Error: {e.output.decode()}"
        # Provide a formatted error tuple consistent with the successful output
        return (os.path.splitext(os.path.basename(file_1))[0],
                os.path.splitext(os.path.basename(file_2))[0],
                "error",  # Placeholder for Distance
                "error",  # Placeholder for P-value
                "error",  # Placeholder for Shared-hashes
                "error")  # Placeholder for Identity percentage
    
def process_genomes(genomes, create_sketch=True, batch_size = 500):
    temp_dir = '/home/alvaroguleon/temp_mash'
    sketch_files = []
    
    # Check if temp_dir already exists and has files
    if os.path.exists(temp_dir) and os.path.isdir(temp_dir):
        # Check if the directory has sketch files or fasta files depending on create_sketch flag
        existing_files = os.listdir(temp_dir)
        for genome_id in genomes.keys():
            expected_file = f"{genome_id}.msh" if create_sketch else f"{genome_id}.fna"
            if expected_file in existing_files:
                sketch_files.append(os.path.join(temp_dir, expected_file))
            else:
                # If expected files are not found, they need to be created
                print(f"Expected file {expected_file} not found in {temp_dir}.")
                # Set flag to indicate file creation is required
                create_files = True
                break
        else:
            # Set flag to indicate no file creation is required if all files were found
            create_files = False
    else:
        # If the directory doesn't exist, create it and set flag to create files
        os.makedirs(temp_dir, exist_ok=True)
        create_files = True

    if create_files:
        print("FASTA files are being created")

        # Create FASTA or sketch files if needed
        for genome_id, sequence in genomes.items():
            file_path = os.path.join(temp_dir, f"{genome_id}.fna")
            with open(file_path, 'w') as file:
                file.write(f">{genome_id}\n{sequence}\n")
            
            if create_sketch:
                sketch_prefix = file_path.replace('.fna', '')  # Remove the '.fna' for mash output prefix
                print(f"Sketch files are being created for {sketch_prefix} {file_path}")
                unix_call(f"mash sketch -o {sketch_prefix} {file_path}")
                sketch_files.append(sketch_prefix + '.msh')  # Append the .msh path
            else:
                sketch_files.append(file_path)  # Append the .fna path

    pairs = [(sketch_files[i], sketch_file) for i in range(len(sketch_files)) for sketch_file in sketch_files[i + 1:]]
    distance_results = []

    with ProcessPoolExecutor() as executor, open('../data/interim/clustering/mash_distances.csv', 'w') as output_file:
        output_file.write('Genome1,Genome2,Distance,P-value,Shared-hashes,Identity\n')  # Update the header with the new column
        for batch_start in range(0, len(sketch_files), batch_size):
            batch_end = min(batch_start + batch_size, len(sketch_files))
            batch_pairs = [(sketch_files[i], sketch_file) for i in range(batch_start, batch_end) for sketch_file in sketch_files[i+1:batch_end]]
            
            futures = {executor.submit(mash_dist_pair, pair[0], pair[1]): pair for pair in batch_pairs}
            for future in as_completed(futures):
                file_1, file_2, distance, p_value, shared_hashes, identity = future.result()  # Update to receive identity
                if not distance.startswith("Error"):  # Only write if no error occurred
                    # Write including the new identity percentage
                    output_file.write(f"{file_1},{file_2},{distance},{p_value},{shared_hashes},{identity:.2f}\n")

            # Print the completion message for the batch
            print(f"Completed processing batch from {batch_start} to {batch_end - 1}")

    distance_df = pd.DataFrame(distance_results)
    shutil.rmtree(temp_dir)  # Clean up temporary directory
    return distance_df

# Example call to the function
#distances = process_genomes(genomes, create_sketch=True, batch_size = 10)
# distances

Completed processing batch from 0 to 9
Completed processing batch from 10 to 19
Completed processing batch from 20 to 29
Completed processing batch from 30 to 39
Completed processing batch from 40 to 49
Completed processing batch from 50 to 59
Completed processing batch from 60 to 69
Completed processing batch from 70 to 79
Completed processing batch from 80 to 89
Completed processing batch from 90 to 99
Completed processing batch from 100 to 109
Completed processing batch from 110 to 119
Completed processing batch from 120 to 129
Completed processing batch from 130 to 139
Completed processing batch from 140 to 149
Completed processing batch from 150 to 159
Completed processing batch from 160 to 169
Completed processing batch from 170 to 179
Completed processing batch from 180 to 189
Completed processing batch from 190 to 199
Completed processing batch from 200 to 209
Completed processing batch from 210 to 219
Completed processing batch from 220 to 229
Completed processing batch from 2

""


In [34]:
distances = pd.read_csv("../data/interim/clustering/mash_distances.csv")
distances

,Genome1,Genome2,Distance,P-value,Shared-hashes,Identity
0,MN335248,MK250021,1.000000,1.0,0/1000,0.0
1,MN335248,MK250022,1.000000,1.0,0/1000,0.0
2,MN335248,MK250024,1.000000,1.0,0/1000,0.0
3,MN335248,MK250028,1.000000,1.0,0/1000,0.0
4,MN335248,MK250026,1.000000,1.0,0/1000,0.0
...,...,...,...,...,...,...
50173,EF380010,DQ490056,1.000000,1.0,0/1000,0.0
50174,EF380010,EF380009,0.004737,0.0,827/1000,82.7
50175,AJ344259,AJ550635,1.000000,1.0,0/1000,0.0
50176,AJ414696,AJ550635,1.000000,1.0,0/1000,0.0


# Network X

In [26]:
import pandas as pd
import networkx as nx
from networkx.algorithms import community
import matplotlib.pyplot as plt


In [6]:
df = pd.read_csv("../data/processed/model_data.csv")
df = df[["Accession", "staining"]]
df

,Accession,staining
0,MN335248,negative
1,MK250029,negative
2,MK250028,negative
3,MK250027,negative
4,MK250026,negative
...,...,...
11151,EF380009,negative
11152,DQ490056,positive
11153,AJ414696,negative
11154,AJ344259,negative


In [27]:
# Assuming 'distances_df' is your DataFrame containing the distance and identity information
distances_df = pd.read_csv('../data/interim/clustering/mash_distances.csv')  # Load your CSV file

# Create a new graph
G = nx.Graph()

# Add edges to the graph
for index, row in distances_df.iterrows():
    # Inverse the identity value to use it as 'distance' if needed
    # Or you can directly use the identity value depending on how you want to cluster
    distance = 1 - row['Identity'] / 100  # Convert percentage to a float value between 0 and 1
    G.add_edge(row['Genome1'], row['Genome2'], weight=distance)

# Now G contains your graph with weighted edges based on sequence identity
# You can now use various clustering or community detection algorithms provided by NetworkX

# For example, to get clusters using the greedy modularity community detection algorithm:

communities = community.greedy_modularity_communities(G, weight='weight')
# This will return a list of sets, where each set is a community of nodes (sequences)

# To see the communities:
for i, community_set in enumerate(communities, start=1):
    print(f"Community {i}: {community_set}")

Community 1: frozenset({'MK250022', 'MK250029', 'MK250027', 'MK250025', 'MK250023', 'MN335248', 'MK250028', 'MK250021', 'MK250024', 'MK250026'})
Community 2: frozenset({'MK250019', 'MK250018', 'MK250020', 'MG592615', 'MK250016', 'MG592455', 'MK250017', 'MG592402', 'MK250015', 'MG592427'})
Community 3: frozenset({'OR413345', 'HM208303', 'MG592394', 'OR204650', 'OR204649', 'OR204652', 'AF334111', 'HM066936', 'OR204651', 'OR413347'})
Community 4: frozenset({'ON212266', 'ON470617', 'OR413344', 'OR290970', 'AY129330', 'MN923124', 'OQ451773', 'OQ745667', 'OR413342', 'MZ892903'})
Community 5: frozenset({'KY385381', 'JX262223', 'AY129332', 'JX262221', 'JX262224', 'JX262225', 'JX262222', 'KY385384', 'EU826469', 'AY129336'})
Community 6: frozenset({'DQ398052', 'JX262215', 'JX262218', 'EU826471', 'JX262217', 'JX262219', 'EU203571', 'EU826470', 'JX262220', 'JX262216'})
Community 7: frozenset({'EU816588', 'EU816589', 'EU816591', 'EU744251', 'EU744252', 'EU826466', 'EU826468', 'EU826467', 'EU770221'

In [21]:
# Function to find to which community (index) a given entry belongs
def find_community(entry, communities_list):
    for index, community in enumerate(communities_list):
        if entry in community:
            return f"Community {index + 1}"  # +1 because index in Python starts at 0
    return "Entry not found in any community"


# Example usage:
entry = 'OR413344' # Replace this with any entry you're searching for
community = find_community(entry, communities)
print(f"The entry {entry} belongs to {community}")


The entry OR413344 belongs to Community 4


In [ ]:
len(communities)

In [ ]:
# Too long
'''plt.figure(figsize=(20, 20))

# Assuming 'G' is your original graph and 'communities' is a list of communities
# Let's say you only want to display the first 10 communities
number_of_communities_to_display = len(communities)

# Create a new graph for visualization purposes
G_vis = nx.Graph()

# Add nodes and edges from the first 10 communities to G_vis
for community_number, community in enumerate(communities):
    if community_number >= number_of_communities_to_display:
        break  # Stop after adding the desired number of communities
    # Add nodes and edges from this community to G_vis
    for node in community:
        G_vis.add_node(node)  # Add the node to the visualization graph
        for neighbor in G.neighbors(node):  # Add an edge within the community in G_vis
            if neighbor in community:
                G_vis.add_edge(node, neighbor)

# Compute the position of each node using the spring layout for the smaller graph G_vis
pos = nx.spring_layout(G_vis)  # Compute the layout

# Draw the nodes and edges for the visualization graph
nx.draw_networkx_edges(G_vis, pos, edge_color="blue", alpha=0.5)
nx.draw_networkx_nodes(G_vis, pos, node_size=50, alpha=0.5, node_color="red")

# Label each community with its number
for community_number, community in enumerate(communities):
    if community_number >= number_of_communities_to_display:
        break  # Stop after adding the desired number of labels
    # Choose one node to label the community (e.g., the first node)
    representative_node = next(iter(community))  # Get a 'representative' node
    plt.text(pos[representative_node][0], pos[representative_node][1], s=str(community_number+1),
             horizontalalignment='center', verticalalignment='center',
             bbox=dict(facecolor='white', alpha=0.6, edgecolor='black', boxstyle='round,pad=0.5'))

plt.axis('off')  # Turn off the axis
plt.show()  # Display the plot'''


In [ ]:
# Use a fixed seed for the layout to be reproducible
seed = 42  # Can be any number, using 42 for example purposes

# Calculate positions for all nodes to be used in plotting
pos = nx.spring_layout(G, seed=seed)  # Pass the seed here

# Setup figure
plt.figure(figsize=(20, 20))

# Draw all edges first to avoid redrawing
nx.draw_networkx_edges(G, pos, alpha=0.1, edge_color='gray')

# Draw nodes using a default color
nx.draw_networkx_nodes(G, pos, node_size=50, alpha=0.5, node_color='green')  # Use 'green' or any color you prefer

# Draw community labels
for community_index, community_nodes in enumerate(communities, start=1):
    # Get a 'representative' node for label placement (for simplicity, we use the first node in the set)
    representative_node = next(iter(community_nodes))
    # Add a label positioned at the representative node's location
    plt.text(pos[representative_node][0], pos[representative_node][1], str(community_index),
             horizontalalignment='center', verticalalignment='center',
             bbox=dict(facecolor='white', alpha=0.6, edgecolor='black', boxstyle='round,pad=0.5'))

plt.axis('off')
plt.title('Network of Genomes')
plt.show()

Color code by staining class

In [22]:
df = pd.read_csv("../data/processed/model_data.csv")
df = df[["Accession", "staining"]]
staining_map = df.set_index('Accession')['staining'].to_dict()

# Determine the majority class for each community
community_staining = {}
for community_number, community in enumerate(communities, start=1):
    # Count the occurrences of each staining within the community
    staining_counts = {'positive': 0, 'negative': 0}
    for node in community:
        # Increment the count for the staining class of this node
        if node in staining_map:  # Only count if the node's staining class is known
            staining_counts[staining_map[node]] += 1
    # Determine the majority class (default to 'negative' if tie)
    majority_staining = max(staining_counts, key=staining_counts.get, default='negative')
    community_staining[community_number] = majority_staining

community_staining

{1: 'negative',
 2: 'negative',
 3: 'negative',
 4: 'negative',
 5: 'positive',
 6: 'positive',
 7: 'positive',
 8: 'positive',
 9: 'positive',
 10: 'negative',
 11: 'negative',
 12: 'negative',
 13: 'negative',
 14: 'positive',
 15: 'negative',
 16: 'negative',
 17: 'negative',
 18: 'negative',
 19: 'negative',
 20: 'positive',
 21: 'negative',
 22: 'positive',
 23: 'positive',
 24: 'positive',
 25: 'positive',
 26: 'positive',
 27: 'positive',
 28: 'negative',
 29: 'negative',
 30: 'negative',
 31: 'negative',
 32: 'negative',
 33: 'negative',
 34: 'negative',
 35: 'negative',
 36: 'positive',
 37: 'positive',
 38: 'negative',
 39: 'negative',
 40: 'negative',
 41: 'negative',
 42: 'positive',
 43: 'negative',
 44: 'negative',
 45: 'negative',
 46: 'positive',
 47: 'negative',
 48: 'negative',
 49: 'negative',
 50: 'positive',
 51: 'positive',
 52: 'negative',
 53: 'negative',
 54: 'positive',
 55: 'positive',
 56: 'negative',
 57: 'negative',
 58: 'negative',
 59: 'negative',
 60: '

In [ ]:
'''# Use a fixed seed for the layout to be reproducible
seed = 42  # Can be any number, using 42 for example purposes

# Calculate positions for all nodes to be used in plotting
pos = nx.spring_layout(G, seed=seed)  # Pass the seed here

# Setup figure
plt.figure(figsize=(20, 20))

# Draw all edges first to avoid redrawing
nx.draw_networkx_edges(G, pos, alpha=0.1, edge_color='gray')

# Draw nodes in batches based on community staining
for community_index, community_nodes in enumerate(communities, start=1):
    color = 'blue' if community_staining[community_index] == 'positive' else 'red'
    nx.draw_networkx_nodes(G, pos, nodelist=list(community_nodes), node_color=color, node_size=50, alpha=0.5)

# Draw community labels
for community_index, community_nodes in enumerate(communities, start=1):
    # Get a 'representative' node for label placement (for simplicity, we use the first node in the set)
    representative_node = next(iter(community_nodes))
    # Add a label positioned at the representative node's location
    plt.text(pos[representative_node][0], pos[representative_node][1], str(community_index),
             horizontalalignment='center', verticalalignment='center',
             bbox=dict(facecolor='white', alpha=0.6, edgecolor='black', boxstyle='round,pad=0.5'))

plt.axis('off')
plt.title('Network of Genomes by Gram Staining Classification')
plt.show()
'''

In [ ]:
# Too long
'''
# Create figure
plt.figure(figsize=(20, 20))

# Assuming 'G' is the original graph and 'communities' is the list of communities
# Let's say you want to display all communities
number_of_communities_to_display = len(communities)

# Assuming community_staining is a dictionary you created earlier, which holds the majority class of staining for each community
# Example: community_staining = {1: 'positive', 2: 'negative', ...}

# Add nodes and edges from the communities to the visualization graph G_vis
for community_number, community in enumerate(communities, start=1):
    if community_number > number_of_communities_to_display:  # Adjust based on how many communities you want to show
        break
    # Determine node color based on majority staining class for the community
    node_color = 'blue' if community_staining[community_number] == 'positive' else 'red'
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, nodelist=community, node_color=node_color, node_size=50, alpha=0.5)
    # Draw edges within the community
    for node in community:
        for neighbor in G.neighbors(node):  # Add an edge within the community
            if neighbor in community:
                nx.draw_networkx_edges(G, pos, edgelist=[(node, neighbor)], width=1, alpha=0.5, edge_color='gray')

# Labeling communities: Place a label at the centroid of each community
for community_number, community in enumerate(communities, start=1):
    if community_number > number_of_communities_to_display:
        break
    # Calculate centroid position as the mean of the positions of the nodes in the community
    centroid_position = [pos[n] for n in community]
    centroid = (sum(x for x, y in centroid_position) / len(centroid_position), 
                sum(y for x, y in centroid_position) / len(centroid_position))
    # Add community label
    plt.text(centroid[0], centroid[1], s=f"Community {community_number}", 
             horizontalalignment='center', verticalalignment='center',
             bbox=dict(facecolor='white', alpha=0.8, edgecolor='black', boxstyle='round,pad=0.3'))

plt.axis('off')
plt.title('Network of Genomes colored by Gram Staining Classification')
plt.show()'''

# Obtaining training and testing data

80% of the clusters should go to training and the remaining 20% to testing data. 2 isolated clusters will be used as validation